# General Exercise Information
Welcome to the exercise for **Machine Learning for 3D Geometry**! 🙂

Some administrative things before you begin with the first exercise:

- You have to submit and pass all 3 exercises to get the 0.3 exam bonus
- You can either work on an exercise alone or team up with one other student; if you work in a team, only one of you has to hand in your solution but make sure to name the submission correctly (see next bullet point)
- Name your submission as your matriculation number(s) separated by underscores, e.g. "0123456_0123457.zip"
- To successfully complete an exercise, fill out the blanks in this jupyter notebook and all referenced files. Make sure all cells in this notebook run as expected. Then, submit everything, including files you generated during the exercise in a single zip file via the submission feature in Moodle
- For each exercise, you have two weeks of working time. Note the deadline date on top of each notebook
- We will upload and present the solutions to each exercise after the respective deadline
- If you have questions, feel free to ask them in the Moodle forum or directly contact us via email at christian.diller@tum.de and can.guemeli@tum.de

# Exercise 1: Surface Representations and Shape Alignment

**Submission Deadline**: 09.11.2022, 23:55

In this exercise, we will go over a few basic geometric tasks including hand-crafted shape generation and conversion between representations as well as simple shape alignment. It is fundamental to understand how these work before we dive into various machine learning tasks in the next two exercises. Being able to solve these tasks will also make working on your projects a lot easier later on.

First of all, make sure you installed all necessary requirements. They are specified in `pyproject.toml`, an easy way to install all of them in a new virtual environment is to use [poetry](https://python-poetry.org), have it install everything (`poetry install`) and run this notebook via `poetry run jupyter notebook`.

The following imports should then work:

In [56]:
from google.colab import drive
import os

gdrive_path='/content/gdrive/MyDrive/TUM/ML3D/E1'

# This will mount your google drive under 'MyDrive'
drive.mount('/content/gdrive', force_remount=True)
# In order to access the files in this notebook we have to navigate to the correct folder
os.chdir(gdrive_path)
# Check manually if all files are present
print(sorted(os.listdir()))

Mounted at /content/gdrive
['.DS_Store', '.gitignore', '.ipynb_checkpoints', 'exercise_1', 'exercise_1.ipynb', 'pyproject.toml', 'sphere_sdf.ply', 'torus_sdf.ply']


In [57]:
%load_ext autoreload
%autoreload 2
!pip install k3d trimesh pytorch_lightning
# !jupyter nbextension install --py --user k3d
# !jupyter nbextension enable --py --user k3d

from pathlib import Path
import numpy as np
import matplotlib as plt
import k3d
import trimesh
import torch

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 1.1. Generating Signed Distance Fields
In the lecture, the concept of implicitly encoded shapes was introduced.
Here, we are specifically interested in generating a Signed Distance Field representation.

In a Signed Distance Field (SDF), the surface of a shape is represented as the zero-level set of a function which maps point locations to their distance from the closest surface.
In addition, the sign encodes whether a voxel lies within a shape (negative sign) or outside (positive sign).

Let's first take a look at some simple examples of such functions.

### (a) SDF for a Sphere
The visualization below shows on the right a slice through the SDF representation of the sphere on the left (positive values in blue, negative in red, close to zero as gray).

Sphere Mesh | Sphere SDF
:- | :-
<img src="exercise_1/images/sphere_mesh.png" alt="sphere_mesh" style="width: 250px;"/> | <img src="exercise_1/images/sphere_sdf.png" alt="sphere_sdf" style="width: 250px;"/>

In this part of the exercise, we do not yet aim to create a grid of SDF values but instead first focus on the evaluation at sparse 3D point locations.

Fill out the function `signed_distance_sphere` in file `exercise_1/implicit_function.py` such that it describes the SDF of a sphere at any given point. Make sure that the results you get in the cell below match the expected results as described in the comments.

Hint: Distances to the surface of a sphere centered at (0, 0, 0) with radius **r** can be defined as $$f(x, y, z) = \sqrt{x^2 + y^2 + z^2} - r$$

In [58]:
from exercise_1.implicit_function import signed_distance_sphere

sphere_radius = 0.35
sphere_center = [0, 0, 0]

# sdf value at point [0, 0, 0], i.e. at the center of the sphere should have a negative sdf
x, y, z = 0, 0, 0
print(f'SDF inside the sphere: {signed_distance_sphere(x, y, z, sphere_radius, sphere_center[0], sphere_center[1], sphere_center[2]):.4f}')

# sdf value at point [0, 0, 0.35], i.e. on the surface of the sphere should have sdf value zero
x, y, z = 0, 0, 0.35
print(f'SDF at surface of the sphere: {signed_distance_sphere(x, y, z, sphere_radius, sphere_center[0], sphere_center[1], sphere_center[2]):.4f}')

# sdf value at point [0, 0.4, 0], i.e. outside the sphere should have a positive sdf value
x, y, z = 0, 0.4, 0
print(f'SDF outside the sphere: {signed_distance_sphere(x, y, z, sphere_radius, sphere_center[0], sphere_center[1], sphere_center[2]):.4f}')

# your function should be able to handle numpy arrays as point coordinate inputs
# let's pass all three points for vectorize inference
x = np.array([0, 0, 0])
y = np.array([0, 0, 0.4])
z = np.array([0, 0.35, 0])
print('SDF for given points:', signed_distance_sphere(x, y, z, sphere_radius, sphere_center[0], sphere_center[1], sphere_center[2]))

SDF inside the sphere: -0.3500
SDF at surface of the sphere: 0.0000
SDF outside the sphere: 0.0500
SDF for given points: [-0.35  0.    0.05]


### (b) SDF for a Torus
Analogous to (a), fill out the function `signed_distance_torus` in file `exercise_1/implicit_function.py` such that the function describes the SDF of a torus.

Hint: Distances to the surface of a torus with major radius **R** and minor radius **r** are defined as $$f(x, y, z) = \sqrt{a^2 + y^2} - r, a=\sqrt{x^2+z^2}-R$$

Torus Mesh | Torus SDF (abs(sdf) > 0.1 not shown)
:- | :-
<img src="exercise_1/images/torus_mesh.png" alt="torus_mesh" style="width: 250px;"/> | <img src="exercise_1/images/torus_sdf.png" alt="torus_sdf" style="width: 250px;"/>

In [59]:
from exercise_1.implicit_function import signed_distance_torus

torus_minor_radius = 0.035
torus_major_radius = 0.35
torus_center = [0, 0, 0]

# let's define a few test points: inside the torus (0, 0, 0.35), surface of the torus (0, 0, 0.385) and outside the torus (0, 0, 0)
x = np.array([0, 0, 0])
y = np.array([0, 0, 0])
z = np.array([0.35, 0.385, 0])
print('SDF for given points:', signed_distance_torus(x, y, z, torus_major_radius, torus_minor_radius, torus_center[0], torus_center[1], torus_center[2]))

SDF for given points: [-3.50000000e-02  2.77555756e-17  3.15000000e-01]


### (c) Composition of SDFs
Using (a) and (b), make a function that describes a hydrogen atom like in the visualization below. We'll represent the proton and the electron with spheres, and the orbit of the electron as a thin torus.

Fill out the function `signed_distance_atom` in file `exercise_1/implicit_function.py` to realize such a signed distance field function. 

Hint: We discussed several possible SDF composition operations in the lecture; pick the correct one here.

Atom Mesh | Atom SDF (abs(sdf) > 0.1 not shown)
:- | :-
<img src="exercise_1/images/atom_mesh.png" alt="atom_mesh" style="width: 250px;"/> | <img src="exercise_1/images/atom_sdf.png" alt="atom_sdf" style="width: 250px;"/>

In [60]:
from exercise_1.implicit_function import signed_distance_atom

# size and placement of shapes is specified in the signed_distance_atom function
# points x, y, z = (0, 0, 0), (0, 0, 0.35) should both lie inside the composite shape, while (0, 0, 1.75) outside
x = np.array([0, 0, 0])
y = np.array([0, 0, 0])
z = np.array([0, 0.35, 0.175])
print('SDF for given points:', signed_distance_atom(x, y, z))

SDF for given points: [-0.1   -0.01   0.075]


### (d) SDF Grids and Visualization
Up until now, you only implemented continuous SDF functions and evaluated them at sparse points.

It is usually more practical to represent a shape in a discrete volumetric grid where each voxel contains one SDF value, pointing towards the closest surface. This is what we want to implement here: Use your SDF functions from above to generate a regular voxel grid, then visualize that in 3D.

First, fill out the function `sdf_grid` in file `exercise_1/sdf_grid.py` to generate an SDF grid given an SDF function and resolution (number of voxels in each dimension). You can assume a unit cube (side length 1 in each dimension) that is centered at the shape centers defined in (a) and (b).

Then, use our pre-defined function `visualize_sdf` in `exercise_1/util/visualization.py` to export your grid as a ply file which you can open in Meshlab to look at and move through your generated SDF.

This is also a good time to familiarize yourself with [Meshlab](http://www.meshlab.net/) if you have not done so yet.

Our visualization function outputs color-coded cubes for each voxel in the grid: Small blue cubes for positive values (outside the surface) and bigger red cubes for negative values (inside the surface).
Cubes very close to the surface are colored in white.

In [87]:
from exercise_1.sdf_grid import sdf_grid
from exercise_1.util.visualization import visualize_sdf

# We use a resolution of 32x32x32 for our grid here. 
# You can also try out different values but note that higher resolutions take significantly 
# more time for generating and exporting the grid.
resolution = 32

In [88]:
# Sphere SDF Grid
sdf_func_sphere = lambda x, y, z: signed_distance_sphere(x, y, z, sphere_radius, sphere_center[0], sphere_center[1], sphere_center[2])
sphere_sdf_grid = sdf_grid(sdf_func_sphere, resolution)
visualize_sdf(sphere_sdf_grid, filename=Path.cwd() / 'sphere_sdf.ply')

Creating SDF visualization for 32^3 grid ...
Exported to /content/gdrive/MyDrive/TUM/ML3D/E1/sphere_sdf.ply


In [89]:
# Torus SDF Grid
sdf_func_torus = lambda x, y, z: signed_distance_torus(x, y, z, torus_major_radius, torus_minor_radius, torus_center[0], torus_center[1], torus_center[2])
torus_sdf_grid = sdf_grid(sdf_func_torus, resolution)
visualize_sdf(torus_sdf_grid, filename=Path.cwd() / 'torus_sdf.ply')

Creating SDF visualization for 32^3 grid ...
Exported to /content/gdrive/MyDrive/TUM/ML3D/E1/torus_sdf.ply


In [90]:
# Atom SDF Grid
# Here, your might notice that a resolution of 32x32x32 or lower is not sufficient to represent the outer ring of the atom.
# This is a common problem especially with machine learning methods that are limited to such a low resolution as we will see in future lectures.
atom_sdf_grid = sdf_grid(signed_distance_atom, resolution)
visualize_sdf(atom_sdf_grid, filename=Path.cwd() / 'atom_sdf.ply')

Creating SDF visualization for 32^3 grid ...
Exported to /content/gdrive/MyDrive/TUM/ML3D/E1/atom_sdf.ply


## 1.2. From Signed Distance Fields to Occupancy Grids
We can also represent shapes by encoding occupancy instead of SDF values. Used in a grid structure, this leads us to Occupancy Grids which simply encode for each voxel if it is inside or outside the represented shape. This has certain advantages and disadvantages compared to SDF grids, especially for learned methods as we will see in later lectures.

### (a) Converting SDF Grids to Occupancy Grids
We consider here an Occupancy Grid as a binary 3D grid with value 0 for voxels outside the shape and 1 for voxels inside. Given an SDF of a shape, converting the shape to an occupancy grid is straight forward.

Fill out the function `occupancy_grid` in file `exercise_1/occupancy_grid.py` such that the function returns an Occupancy Grid given an existing SDF Grid.

In [125]:
from google.colab import output
output.enable_custom_widget_manager()

In [122]:
from exercise_1.occupancy_grid import occupancy_grid
from exercise_1.util.visualization import visualize_occupancy, visualize_pointcloud, visualize_mesh

# Here, we use a grid of 128x128x128
resolution = 128

In [124]:
# Sphere Occupancy Grid
sphere_occ_grid = occupancy_grid(sdf_func_sphere, resolution)
visualize_occupancy(sphere_occ_grid)

Output()

In [120]:
# Torus Occupancy Grid
torus_occ_grid = occupancy_grid(sdf_func_torus, resolution)
visualize_occupancy(torus_occ_grid)

Output()

In [107]:
# Atom Occupancy Grid
atom_occ_grid = occupancy_grid(signed_distance_atom, resolution)
visualize_occupancy(atom_occ_grid)

Output()

### (b) Implicit Functions for more complex shapes

The SDF functions you implemented and used so far encode only simple shapes which could be represented by rather straight-forward mathematical functions. What if we need to represent a complex shape, e.g. the stanford bunny? It's hard to come up with an elegant mathematical equation in this case.

One way to figure out such implicit shape function is to <em>learn</em> the function in a supervised way. In later lectures, we'll go over works like DeepSDF [1] and Occupancy Networks [2] which are seminal works showing how to represent and learn SDF/Occupancy functions with deep neural networks.

For now let's try running your occupancy grid extraction method on a learned function to represent a complex shape.

In [108]:
from exercise_1.util.mlp.model import signed_distance_mlp

# runs on cpu, so might be slow
shape_grid = occupancy_grid(signed_distance_mlp, resolution)
visualize_occupancy(shape_grid, flip_axes=True)

RuntimeError: ignored

For those curious on how to train such a function, a minimal sample implementation is provided [here](https://github.com/nihalsid/shape_sdf).

## 1.3. From Signed Distance Fields to Triangle Meshes

Operating on regular voxel grids with either SDFs or Occupancy Grids has many advantages as you will also see in later exercises.

However, eventually we want to be able to extract the shape encoded in an implicit representation into an explicit format that can be viewed more easily and is more widely supported by graphics pipelines. This is where polygon (or more specifially, triangle) meshes come into play.

As presented in the lecture, the Marching Cubes [3] algorithm is one of the most widely used approaches to extract such meshes from implicit representations. In this task, we want to create a naive but fully functional implementation of this algorithm from scratch. You will find that it is surprisingly easy to implement!

Marching Cubes iteratively loops over all cubes defined between the voxels of a regular grid.

For each such cube, it does the following:
- Compute an 8-bit index representing the specific configuration of its corners (storing for each corner whether it lies inside or outside of the shape)
- Using this index, retreive information about where and how triangles intersect the cube edges from a pre-computed table
- Add found vertices and faces to a global list

We will use the following edge and corner indices for our implementation:

<center><img src="exercise_1/images/mc_indices.png" alt="Marching Cubes Indices" style="width: 250px;"/></center>

This visualization also includes the axes specification we use for this exercise: Positive X to the right; Positive Y out of the image plane; Positive Z upwards.

### (a) Compute the Cube Index

Implement the function computing a cube index from the current cube by filling out function `compute_cube_index` in `exercise_1/marching_cubes.py`.

The function takes a cube, represented by a list of SDF values from the corners of the cube. The order is given by the corner indices visualized above.

For each corner, you should set the corresponding bit to 1 if that corner's SDF value is < 0 and to 1 otherwise.

Example: A cube intersects the surface such that its top half lies within the shape (= top 4 corner voxels hold negative SDF values) and the bottom half outside (= bottom 4 corner voxels hold positive SDF values).

Its index should then be 11110000 = 240.

Note that corner 0 in your list of SDF values modifies the least significant bit.

In [ ]:
from exercise_1.marching_cubes import compute_cube_index

cube = np.array([-1, -1, -1, -1, 1, 1, 1, 1])
index = compute_cube_index(cube)
print(f"Cube Index for {cube} = {index:08b}(={index})")

# This is the example from above
cube = np.array([1, 1, 1, 1, -1, -1, -1, -1])
index = compute_cube_index(cube)
print(f"Cube Index for {cube} = {index:08b}(={index})")

# Take a look at the triangle_table. How many triangles are defined for this index?
cube = np.array([1, 1, 1, 1, 1, 1, 1, 1])
index = compute_cube_index(cube)
print(f"Cube Index for {cube} = {index:08b}(={index})")

### (b) Implement Marching Cubes
Now that we have the index, all that is left to do is to loop over all possible cubes and to use the pre-defined triangle table that tells us which edges are intersected in the current cube. Indexing the triangle table with the cube index gives us between 0 and 5 edge index triples. We use each such triple to create a face. 

A (triangular) face is defined as a tuple of 3 integer indices with each index referring to a vertex in a corresponding vertex list. The minimal example in exercise part 1.4 gives an example for that: A face is defined in line 4 which references vertices 1, 2, and 3 defined in the vertex list in lines 1-3.

In our case, we get the face vertices by placing a point on each edge that is referenced in our edge index triple. Use the pre-defined function `vertex_interpolation` to place that point halfway between the corners adjacent to the edge. Use the function as-is right now, we will modify it in part (c).

Implement function `marching_cubes` in `exercise_1/marching_cubes.py` which takes an sdf grid as input and returns a tuple of 2 numpy arrays, the first for the vertex locations and the second for face definitions.

You don't need to find and remove vertex duplicates for this naive implementation; this is something you would optimize when going for an ideal implementation, though.

In [ ]:
from exercise_1.marching_cubes import marching_cubes
from exercise_1.util.visualization import visualize_mesh

sdf_grid_sphere = sdf_grid(sdf_func_sphere, 32)
vertices, faces = marching_cubes(sdf_grid_sphere)
visualize_mesh(vertices, faces)

### (c) Vertex Interpolation

Your output should look like the left image in the visualization below.

One thing we still have to take care of is to reduce the effect of the grid structure. In your implementation so far, the vertex locations were set to lie exactly half way between the two corners of an edge.

One trick to reduce the amount of grid artifacts in our mesh is to interpolate the vertex locations based on their relative SDF values since each such value tells us precisely where the surface should lie.

Change the implementation of function `vertex_interpolation` in file `exercise_1/marching_cubes.py` to interpolate the vertex location based on corner points p_1 and p_2 as well as their corresponding SDF values v_1 and v_2.

Run your Marching Cubes implementation again with the modified function below. Your output should now look much smoother, like on the right side of the visualization below.

No Interpolation | Interpolation
:- | :-
<img src="exercise_1/images/mc_no_interpolation.png" alt="no interpolation" style="width: 250px;"/> | <img src="exercise_1/images/mc_interpolation.png" alt="interpolation" style="width: 250px;"/>

In [ ]:
vertices, faces = marching_cubes(sdf_grid_sphere)
visualize_mesh(vertices, faces)

You can also play around with your torus and atom shapes below.

Try out the effects of different resolutions and how vertex interpolation affects things there.

In [ ]:
sdf_grid_sphere = sdf_grid(sdf_func_sphere, 128)
vertices, faces = marching_cubes(sdf_grid_sphere)
visualize_mesh(vertices, faces)

In [ ]:
sdf_grid_torus = sdf_grid(sdf_func_torus, 64)
vertices, faces = marching_cubes(sdf_grid_torus)
visualize_mesh(vertices, faces)

In [ ]:
# Note that the orbit of the atom is not preserved at 64^3 resolution or lower, try going higher in resolution, e.g. 128 (will be much slower)
sdf_grid_atom = sdf_grid(signed_distance_atom, 64)
vertices, faces = marching_cubes(sdf_grid_atom)
visualize_mesh(vertices, faces)

In [ ]:
# runs on cpu, so might be slow
sdf_grid_mlp = sdf_grid(signed_distance_mlp, 64)
vertices, faces = marching_cubes(sdf_grid_mlp)
visualize_mesh(vertices, faces, flip_axes=True)

## 1.4. Writing Triangle Meshes to Disk as Wavefront OBJ

Triangle meshes are simple data structures that can easily be stored in common data formats like OBJ.

In this format, each line simply starts with a _v_ if it defines a vertex or _f_ if it defines a face.
The letter is then followed by three float values (written as strings) denoting the x, y, and z locations for a vertex or three integer values denoting the indices of the vertices that are part of this triangle face. All values are separated by a space. Note that vertices are 1-indexed in the official obj specification.

Minimal example:
```
v 1.0 2.0 3.0
v 2.0 3.0 4.0
v 3.0 4.0 5.0
f 1 2 3
```

Implement the function `export_mesh_to_obj` in file `exercise_1/export.py` to write the list of vertices you got from Marching Cubes, together with the list of faces. The resulting file should be viewable in Meshlab.

In [ ]:
from exercise_1.export import export_mesh_to_obj

vertices, faces = marching_cubes(sdf_grid_sphere)
export_mesh_to_obj("exported_mesh.obj", vertices, faces)

## 1.5. From Triangle Meshes to Point Clouds

Point clouds are another 3D representation popular because of their simplicity and efficiency, and are quite often the output format provided by 3D capturing devices. In the lecture we discussed one way to sample point clouds from meshes by sampling uniformly across the mesh surface.

This can be done by first determining the number of points we need to sample for each triangle, based on its surface area. Then, we sample that number of points using barycentric coordinates for each triangle by generating random numbers between 0 and 1 for r_1 and r_2 and then apply the formulas $$u=1-\sqrt{r_1}, v=\sqrt{r_1}(1-r_2), w=\sqrt{r_1}r_2, P=uA+vB+wC$$ to get the coordinates of the sampled point **P**.

Fill in the function `sample_point_cloud` in file `exercise_1/point_cloud.py` to realize such a sampling strategy.

Hint: An easy way to sample based on surface area is to calculate the probability of sampling a triangle and then use `np.random.choice`.

In [ ]:
from exercise_1.point_cloud import sample_point_cloud

n_points = 5000

sdf_grid_atom = sdf_grid(signed_distance_atom, 64)
vertices, faces = marching_cubes(sdf_grid_atom)
sampled_points = sample_point_cloud(vertices, faces, n_points)
visualize_pointcloud(sampled_points, .25)

MeshLab supports viewing point clouds stored as OBJ files. Since we don't have any faces, we can store only the vertices in the OBJ. 

Fill out the function `export_pointcloud_to_obj` in `exercise_1/export.py` and export the point cloud to disk. You can re-use parts of the export function you wrote before here.

In [ ]:
from exercise_1.export import export_pointcloud_to_obj

# export to "exported_pointcloud.obj", check out the mesh in MeshLab
export_pointcloud_to_obj("exported_pointcloud.obj", sampled_points)

## 1.6. Rigid Shape Aligment with Procrustes

In the second lecture, you learned about shape alignment using Procrustes and Iterative Closest Point and shape description using various techniques.

In this last part of the exercise, we will cover a simple method to align two shapes together: Solving the Orthogonal Procrustes problem [4]. If you want to learn more about shape description and alignment, we highly recommed taking the 3D Scanning & Motion Capture lecture as those topics are the focus of that lecture.

Here we'll try to align the following two shapes, given near-perfect point correspondences between the shapes. Note that when working with real data, more involved methods like ICP are required since neither correspondences nor shape overlap will be perfect.

Executing the cell below shows a visualization of the two shapes in their current state.

In [ ]:
from exercise_1.util.visualization import visualize_shape_alignment
visualize_shape_alignment()

Fill out the function `procrustes_align` in `exercise_1/alignment.py` such that this function returns the rigid transformation from input (red) to target (green). It should return a rotation R and translation t which you can then use to visualize the final shape alignment using our pre-defined function `visualize_shape_alignment`.

The function already takes in the correspondences, so you don't have to worry about that.

For the given meshes and correspondences you should be able to get close to 0 aligment loss and almost perfect aligment of the meshes.

In [ ]:
from exercise_1.alignment import procrustes_align, load_correspondences
from exercise_1.util.visualization import visualize_shape_alignment

# load correspondences
correspondences_x, correspondences_y = load_correspondences()
# align correspondences, in case of perfect aligment should be very close to zero (< 1e-8)
R, t = procrustes_align(correspondences_x, correspondences_y)

# perfect aligment would cause perfect superimposition of the two meshes
# perfect aligment (on my system) appears as green mesh with gliching red colors when changing the viewing angle
visualize_shape_alignment(R, t)

## Submission

This is the end of exercise 1 🙂. Please create a zip containing all files we provided, everything you modified, and all of your generated output/visualization files. Name it with your matriculation number(s) as described on top. Make sure this notebook can be run without problems. Then, submit via Moodle.

**Submission Deadline**: 09.11.2022 23:55

## References

[1] Park, Jeong Joon, et al. "Deepsdf: Learning continuous signed distance functions for shape representation." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019

[2] Mescheder, Lars, et al. "Occupancy networks: Learning 3d reconstruction in function space." Proceedings of the IEEE/CVF Conference on Computer Vision and Pattern Recognition. 2019.

[3] Lorensen, William E., and Harvey E. Cline. "Marching cubes: A high resolution 3D surface construction algorithm." ACM siggraph computer graphics 21.4 (1987): 163-169.

[4] Schönemann, Peter H. "A generalized solution of the orthogonal procrustes problem." Psychometrika 31.1 (1966): 1-10.

